In [4]:
# !pip install keras-tuner
# !pip install pydot graphviz
# !pip install scikit-optimizer

In [5]:
from google.colab import drive
drive.mount('/content/drive')
import sys
sys.path.append('/content/drive/My Drive/rps2')

from Mi_ML import *
# desde cero hacemos uno nuevo. la logica de programacion mas engorrosa ya esta dentro de fucniones a compilar
# por lo que nos centraremos en terminar un modelo simple y luego a posterior lo empaquetamos en una clase con utilidades, otra nueva con muchas funciones a reutilizar
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split #TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
%matplotlib inline
from skopt import gp_minimize
from skopt.space import Real, Integer
from skopt.utils import use_named_args




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
from RPS_game import *
from RPS1 import *

# player kris

In [7]:
def player_kin(prev_play, counter=[0]):

    counter[0] += 1
    choices = ["S", "P", "R"]
    return choices[counter[0] % len(choices)]

# play_mod

In [8]:
def play_mod(player1, player2, num_games, verbose=False):
    p1_prev_play = ""
    p2_prev_play = ""
    results = {"p1": 0, "p2": 0, "tie": 0}

    p1_list = []
    p2_list = []

    for _ in range(num_games):
        p1_play = player1(p2_prev_play)
        p2_play = player2(p1_prev_play)

        if p1_play == p2_play:
            results["tie"] += 1
            winner = "Tie."
        elif (p1_play == "P" and p2_play == "R") or (
                p1_play == "R" and p2_play == "S") or (p1_play == "S"
                                                       and p2_play == "P"):
            results["p1"] += 1
            winner = "Player 1 wins."
        elif p2_play == "P" and p1_play == "R" or p2_play == "R" and p1_play == "S" or p2_play == "S" and p1_play == "P":
            results["p2"] += 1
            winner = "Player 2 wins."

        if verbose:
            print("Player 1:", p1_play, "| Player 2:", p2_play)
            print(winner)
            print()

        p1_list.append(p1_play)
        p2_list.append(p2_play)

        p1_prev_play = p1_play
        p2_prev_play = p2_play

    games_won = results['p2'] + results['p1']

    if games_won == 0:
        win_rate = 0
    else:
        win_rate = results['p1'] / games_won * 100



    print("Final results:", results)
    print(f"Player 1 win rate: {win_rate}%")
    print("Lista de jugadas p1: ", p1_list)
    print("Lista de jugadas p2: ", p2_list)

    return (win_rate, p1_list, p2_list)

In [9]:
win, p1, p2 = play_mod(player_kin, kris,4000, verbose=True) # creo que asi esta bien para ganarle

Se han truncado las últimas 5000 líneas del flujo de salida.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | Player 2: P
Player 1 wins.

Player 1: P | Player 2: R
Player 1 wins.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | Player 2: P
Player 1 wins.

Player 1: P | Player 2: R
Player 1 wins.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | Player 2: P
Player 1 wins.

Player 1: P | Player 2: R
Player 1 wins.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | Player 2: P
Player 1 wins.

Player 1: P | Player 2: R
Player 1 wins.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | Player 2: P
Player 1 wins.

Player 1: P | Player 2: R
Player 1 wins.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | Player 2: P
Player 1 wins.

Player 1: P | Player 2: R
Player 1 wins.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | Player 2: P
Player 1 wins.

Player 1: P | Player 2: R
Player 1 wins.

Player 1: R | Player 2: S
Player 1 wins.

Player 1: S | 

In [10]:
# import numpy as np
# from sklearn.preprocessing import OneHotEncoder

# jugadas_p1 = p1
# jugadas_p2 = p2

# # Codificar las jugadas
# jugadas = {'P': 0, 'R': 1, 'S': 2}
# jugadas_p1 = [jugadas[jugada] for jugada in jugadas_p1]
# jugadas_p2 = [jugadas[jugada] for jugada in jugadas_p2]

# # Convertir a arrays de numpy
# jugadas_p1 = np.array(jugadas_p1)
# jugadas_p2 = np.array(jugadas_p2)

# # Crear secuencias para entrenamiento
# secuencia_length = 3  # Ejemplo de longitud de secuencia
# X = []
# y = []

# for i in range(len(jugadas_p1) - secuencia_length):
#     X.append(jugadas_p1[i:i+secuencia_length])
#     y.append(jugadas_p2[i+secuencia_length])

# X = np.array(X)
# y = np.array(y)

# # One-hot encoding
# encoder = OneHotEncoder(sparse_output=False) # sparse anteriormente, pero me daba error
# y = encoder.fit_transform(y.reshape(-1, 1))

# # Asegúrate de que X tenga la forma (samples, time steps, features)
# X = X.reshape(X.shape[0], X.shape[1], 1)


# -------------
# ahora le jugador p2 sera el de las ejemplo y p1 el de las etiquetas

import numpy as np
from sklearn.preprocessing import OneHotEncoder

jugadas_p1 = p1
jugadas_p2 = p2

# Codificar las jugadas
jugadas = {'P': 0, 'R': 1, 'S': 2}
jugadas_p1 = [jugadas[jugada] for jugada in jugadas_p1]
jugadas_p2 = [jugadas[jugada] for jugada in jugadas_p2]

# Convertir a arrays de numpy
jugadas_p1 = np.array(jugadas_p1)
jugadas_p2 = np.array(jugadas_p2)

# Crear secuencias para entrenamiento
sec = 2
secuencia_length = sec  # Ejemplo de longitud de secuencia
X = []
y = []

for i in range(len(jugadas_p1) - secuencia_length):
    X.append(jugadas_p1[i:i+secuencia_length])
    y.append(jugadas_p1[i+secuencia_length])

X = np.array(X)
y = np.array(y)

# One-hot encoding
encoder = OneHotEncoder(sparse_output=False) # sparse anteriormente, pero me daba error
y = encoder.fit_transform(y.reshape(-1, 1))

# Asegúrate de que X tenga la forma (samples, time steps, features)
X = X.reshape(X.shape[0], X.shape[1], 1)



In [11]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Dropout, BatchNormalization, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt

# Configurar TensorFlow para usar DirectML
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        tf.config.set_visible_devices(physical_devices[0], 'GPU')
    except RuntimeError as e:
        print(e)

%matplotlib inline


In [12]:
import tensorflow.compat.v1 as tf
tf.enable_eager_execution()

# Configurar TensorFlow para usar DirectML
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        tf.config.set_visible_devices(physical_devices[0], 'GPU')
    except RuntimeError as e:
        print(e)


In [13]:
from tensorflow.keras.layers import SimpleRNN

def create_model(optimizer='adam', neurons=400, Dense_neurons=60, d1=0.05, d2=0.11, d3=0.3, learning_rate=0.0002):
    if optimizer == 'adam':
        optimizer = Adam(learning_rate=learning_rate)

    model = Sequential()
    model.add(SimpleRNN(neurons, return_sequences=True, input_shape=(X.shape[1], X.shape[2])))
    model.add(Dropout(d1))
    model.add(SimpleRNN(neurons, return_sequences=True))
    model.add(Dropout(d2))
    model.add(GlobalAveragePooling1D())
    model.add(Dense(Dense_neurons, activation='relu'))
    model.add(Dropout(d3))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = create_model()

# Early stopping para detener el entrenamiento si no hay mejora significativa
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

# Entrenar el modelo
# with tf.device('/device:GPU:0'):
history = model.fit(X, y, epochs=40, batch_size=50, validation_split=0.18, callbacks=[early_stopping])



Epoch 1/40
66/66 [==============================] - 16s 62ms/step - loss: 0.4470 - accuracy: 0.8215 - val_loss: 0.0658 - val_accuracy: 1.0000
Epoch 2/40
66/66 [==============================] - 1s 22ms/step - loss: 0.0326 - accuracy: 1.0000 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 3/40
66/66 [==============================] - 1s 21ms/step - loss: 0.0075 - accuracy: 1.0000 - val_loss: 0.0014 - val_accuracy: 1.0000
Epoch 4/40
66/66 [==============================] - 2s 27ms/step - loss: 0.0044 - accuracy: 0.9997 - val_loss: 7.2562e-04 - val_accuracy: 1.0000
Epoch 5/40
66/66 [==============================] - 2s 33ms/step - loss: 0.0029 - accuracy: 1.0000 - val_loss: 3.1930e-04 - val_accuracy: 1.0000
Epoch 6/40
66/66 [==============================] - 2s 36ms/step - loss: 0.0016 - accuracy: 1.0000 - val_loss: 1.9071e-04 - val_accuracy: 1.0000
Epoch 7/40
66/66 [==============================] - 2s 25ms/step - loss: 0.0014 - accuracy: 1.0000 - val_loss: 1.2524e-04 - val_accuracy: 1.0

In [14]:
model.predict(X)

125/125 [==============================] - 1s 7ms/step


array([[1.5471149e-07, 4.0624968e-08, 9.9999970e-01],
       [9.9999899e-01, 7.6628339e-07, 2.1212443e-07],
       [1.4782454e-07, 9.9999970e-01, 7.7083968e-08],
       ...,
       [1.4782455e-07, 9.9999976e-01, 7.7083968e-08],
       [1.5471149e-07, 4.0624968e-08, 9.9999976e-01],
       [9.9999905e-01, 7.6628345e-07, 2.1212446e-07]], dtype=float32)

In [15]:
# o le faltaron epocas ? que ssera ?
# la precicion es alta , pero el modelo se entreno poco, sera que estara mal entrenado. hay que testearlo con los bots

# ajustamos para tener la funcion de player correcpondiente
#import numpy as np
#import tensorflow as tf
# from tensorflow.keras.models import load_model

# Mapeo de índices a etiquetas
label_map = {0: 'P', 1: 'R', 2: 'S'}
#jugadas = {'P': 0, 'R': 1, 'S': 2}

def rps_one_hot_single(play): # acá esta el problema , si mi tamaño de secuencia es variable vas a tener que adptar este algoritmo
    if play == 'P':
        return [1, 0, 0]
    elif play == 'R':
        return [0, 1, 0]
    elif play == 'S':
        return [0, 0, 1]


import numpy as np

# Definir la función IA_player
def IA_player(prev_play, model = model, opponent_history=[], secuencia_length=sec, jugadas={'P': 0, 'R': 1, 'S': 2}):
    if prev_play:
        opponent_history.append(jugadas[prev_play])

    if len(opponent_history) < secuencia_length:
        # Si no tenemos suficientes jugadas para hacer una predicción, devolver una jugada aleatoria
        return np.random.choice(['P', 'R', 'S'])

    # Usar las últimas `secuencia_length` jugadas del oponente para hacer una predicción
    input_sequence = np.array(opponent_history[-secuencia_length:]).reshape(1, secuencia_length, 1)

    # Hacer una predicción con el modelo
    prediction = model.predict(input_sequence, verbose = False)

    # Decodificar la predicción
    predicted_move = np.argmax(prediction)

    # Decodificar la jugada predicha a 'P', 'R' o 'S'
    inv_jugadas = {v: k for k, v in jugadas.items()}
    guess = inv_jugadas[predicted_move]

    return guess[-1:]

# Ejemplo de uso:
# model: El modelo de IA ya entrenado
# prev_play: La última jugada del oponente (por ejemplo, 'P', 'R' o 'S')

# En cada ronda del juego, llama a IA_player:
# next_move = IA_player(prev_play, model)



In [16]:
# a jugar !
from RPS_game import play, mrugesh, abbey, quincy, kris, human, random_player
from RPS1 import player
from unittest import main

with tf.device('/device:GPU:0'):
  # play(AI_player5, quincy, 1000)
  # play(AI_player5, abbey, 1000)
  play(IA_player, kris, 1000)
  # play(AI_player5, mrugesh, 1000)

Final results: {'p1': 500, 'p2': 0, 'tie': 500}
Player 1 win rate: 100.0%


In [17]:
import tensorflow as tf

# Configura TensorFlow para usar DirectML
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
    try:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
    except RuntimeError as e:
        print(e)


In [18]:
# sabemos que gana el 100%